In [73]:
import folium
import pandas as pd

#read in the Google Sheets called route5_on_time_performance
data = pd.read_csv("route5_performance_4_7.csv")
#Goal: for every row, make a tuple of (latitude, longitude) + other details --> giant list of info per stop

other_coordinate_info = []
for index, row in data.iterrows(): 
    tuples = (row['stop_lat'], row['stop_lon'])
    stop = [tuples, row['stop_id'], row['stop_name'], row['difference']]
    other_coordinate_info.append(stop)
    
#other_coordinate_info contain: (lat, long), stop_id, number of times, average on-time, stop_name

In [74]:
#Create a base map using coordinates of Atlanta
m = folium.Map(location=[33.8792, -84.3797], zoom_start = 12)

#add markers
for stop in other_coordinate_info:
    info = 'stop_id: {}, stop_name: {}, avg_ontime_seconds: {}'.format(stop[1], stop[2], stop[3])
    if stop[3] > 600:
        folium.Marker(stop[0], popup = info, icon=folium.Icon(color='black')).add_to(m)
    if ((stop[3] < 600) & (stop[3] >= 300)):
        folium.Marker(stop[0], popup = info, icon=folium.Icon(color='red')).add_to(m)
    if ((stop[3] < 300) & (stop[3] >= 0)): 
        folium.Marker(stop[0], popup = info, icon=folium.Icon(color='white')).add_to(m)
    if ((stop[3] > -300) & (stop[3] < 0)): 
        folium.Marker(stop[0], popup = info, icon=folium.Icon(color='blue')).add_to(m)
    if ((stop[3] > -600) & (stop[3] < -300)): 
        folium.Marker(stop[0], popup = info, icon=folium.Icon(color='orange')).add_to(m)

In [75]:
m.save('route15_180407.html')

In [152]:
from pathlib import Path
import pandas as pd
import numpy as np

import seaborn as sns
import folium
from folium.plugins import TimestampedGeoJson

data_7 = pd.read_csv("route5_performance_4_7.csv")
data_1 = pd.read_csv("route5_performance_4_1.csv")
data_2 = pd.read_csv("route5_performance_4_2.csv")
meta = [data_7, data_1, data_2]



1222
798
1756
> Creating GeoJSON features...
3776


In [163]:
from pathlib import Path
import pandas as pd
import numpy as np

import seaborn as sns
import folium
from folium.plugins import TimestampedGeoJson

data_7 = pd.read_csv("route5_performance_4_7.csv")
data_1 = pd.read_csv("route5_performance_4_1.csv")
data_2 = pd.read_csv("route5_performance_4_2.csv")
meta = [data_7, data_1, data_2]

def prepare_data(meta):
    for df in meta:
        df['actual_time'] =  pd.to_datetime(df['actual_time'])
        for stop in df:
            df.actual_time.unique()

        for index, row in df.iterrows():
            if row['difference'] > 600:
                df.loc[index,'color'] = 'black'
            if ((row['difference'] < 600) & (row['difference'] >= 300)):
                df.loc[index,'color'] = 'red'
            if ((row['difference'] < 300) & (row['difference'] >= 0)): 
                df.loc[index,'color'] = 'white'
            if ((row['difference'] > -300) & (row['difference'] < 0)): 
                df.loc[index,'color'] = 'blue'
            if ((row['difference'] > -600) & (row['difference'] < -300)): 
                df.loc[index,'color'] = 'orange'
    return meta


def create_geojson_features(meta):
    print('> Creating GeoJSON features...')
    features = []
    for df in meta:
        for _, row in df.iterrows():
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[row['stop_lon'],row['stop_lat']]
                },
                'properties': {
                    'time': row['actual_time'].date().__str__(),
                    'style': {'color' : row['color']},
                    'icon': 'circle',
                    'iconstyle':{
                        'fillColor': row['color'],
                        'fillOpacity': 0.8,
                        'stroke': 'true',
                        'radius': 7
                    }
                }
            }
            features.append(feature)
    return features

def make_map(features):
    print('> Making map...')
    coords_roswell=[33.8792, -84.3797]
    bus_map = folium.Map(location=coords_roswell, control_scale=True, zoom_start=12)

    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1D'
        , add_last_point=True
        , auto_play=True
        , loop=True
        , max_speed=.5
        , loop_button=True
        , date_options='M/D/YYYY HH:mm:ss'
        , time_slider_drag_update=True
    ).add_to(bus_map)
    print('> Done.')
    return bus_map

def plot_route(meta):
    new_meta = prepare_data(meta)
    features = create_geojson_features(new_meta)
    return make_map(features)

plot_route(meta)

> Creating GeoJSON features...
> Making map...
> Done.


In [80]:
import folium
from folium import plugins
m = folium.Map(location=[35.68159659061569, 139.76451516151428], zoom_start = 16)

# Lon, Lat order.
lines = [
    {
        'coordinates': [
            [139.76451516151428, 35.68159659061569],
            [139.75964426994324, 35.682590062684206],
        ],
        'dates': [
            '2017-06-02T00:00:00',
            '2017-06-02T00:10:00'
        ],
        'color': 'red'
    },
    {
        'coordinates': [
            [139.75964426994324, 35.682590062684206],
            [139.7575843334198, 35.679505030038506],
        ],
        'dates': [
            '2017-06-02T00:10:00',
            '2017-06-02T00:20:00'
        ],
        'color': 'blue'
    },
    {
        'coordinates': [
            [139.7575843334198, 35.679505030038506],
            [139.76337790489197, 35.678040905014065],
        ],
        'dates': [
            '2017-06-02T00:20:00',
            '2017-06-02T00:30:00'
        ],
        'color': 'green',
        'weight': 15,
    },
    {
        'coordinates': [
            [139.76337790489197, 35.678040905014065],
            [139.76451516151428, 35.68159659061569],
        ],
        'dates': [
            '2017-06-02T00:30:00',
            '2017-06-02T00:40:00'
        ],
        'color': '#FFFFFF',
    },
]

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)

display(m)